<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>Editado por Equipo Docente IIC2233 2018-1 y 2018-2. </font><br>
<font size='1'>Incluye partes de un documento de Nebil Kawas 2017-2.</font><br>
<font size='1'>Incluye partes de un material confeccionado por Karim Pichara y Christian Pieringer en 2015.</font>
<br>
</p>

## Ejemplos de decoradores

### Decorador sin constructor

Acá definimos un decorador que nos permitirá hacer _logging_, cuando llamemos a una función decorada.

In [1]:
import time

def logger(function):
    def wrapper(*args, **kwargs):
        start = time.time()
        print("Ejecutando la función...")
        result = function(*args, **kwargs)
        end = time.time()
        print(f"Finalizando la función. Demoró {end-start:.4f} segundos.")
        
        return result
    return wrapper

### Un decorador con constructor

Acá definimos un nuevo decorador que nos permitirá hacer algo similar al anterior.  
Además, agregamos una funcionalidad para notificar si la ejecución ha demorado más que un _threshold_ específico.

In [2]:
import time 

def timer(threshold=2): # Podemos, incluso, entregarle un parámetro opcional.
    def check_time(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            
            if end - start > threshold:
                print("[Warning] La ejecución ha tomado mucho tiempo")
            
            return result
        return wrapper
    return check_time

Creamos dos funciones de suma: una normal, y otra un poco más somnolienta.  
Y además, aprovechamos de aplicarles distintos decoradores recién definidos.

In [3]:
@timer(1)
def normal_sum(a, b):
    return a + b

@logger
@timer(2)
def sleepy_sum(a, b, c):
    time.sleep(3)
    return a + b + c

In [4]:
print("Resultado:", normal_sum(3, 4))
print()
print()
print("Resultado:", sleepy_sum(4, 5, 6))

Resultado: 7


Ejecutando la función...
[Warning] La ejecución ha tomado mucho tiempo
Finalizando la función. Demoró 3.0035 segundos.
Resultado: 15


### Más aplicaciones

#### Memorización de cómputos


Supongamos que tenemos la siguiente implementación recursiva de la función que retorna los números de Fibonacci.

In [5]:
def fib(n):
    print(f"Calculando el número {n} de Fibonacci")
    if n == 0 or n == 1:
        return n
    else:
        return fib(n - 1) + fib(n - 2)

Veamos qué pasa cuando queremos, por ejemplo, el cuarto número de Fibonacci.

In [6]:
fib(4)

Calculando el número 4 de Fibonacci
Calculando el número 3 de Fibonacci
Calculando el número 2 de Fibonacci
Calculando el número 1 de Fibonacci
Calculando el número 0 de Fibonacci
Calculando el número 1 de Fibonacci
Calculando el número 2 de Fibonacci
Calculando el número 1 de Fibonacci
Calculando el número 0 de Fibonacci


3

Si nos fijamos bien, hemos calculado dos veces el número 2, tres veces el 1 y otras dos veces el número 0. Esto empeora a la hora de pedir números más grandes. Podemos concluir que esta implementación no es muy eficiente, debido a que repetimos cálculos innecesariamente. Para resolver eso, podríamos "memorizar" los números ya calculados de tal manera de no tener que volver a computarlos si ya lo hemos hecho. 

Podemos usar un decorador que tome la función `fib` y le agregue una memoria, que verifique primero si ya hemos calculado un número (y si es así, lo retornamos). En caso de que no lo hayamos calculado, llamamos a la función original para que lo haga.

In [7]:
def memoize(function):
    data = {}
    def wrapper(x):  # Notar que este decorador sólo servirá para funciones con un parámetro.
        if x not in data:
            # En caso de que no hayamos calculado el número x, lo calculamos y guardamos.
            data[x] = function(x)
        return data[x]
    return wrapper

In [8]:
@memoize
def fib(n):
    print(f"Calculando el número {n} de Fibonacci")
    if n == 0 or n == 1:
        return n
    else:
        return fib(n - 1) + fib(n - 2)

Ahora, veamos que sucede.

In [9]:
fib(4)

Calculando el número 4 de Fibonacci
Calculando el número 3 de Fibonacci
Calculando el número 2 de Fibonacci
Calculando el número 1 de Fibonacci
Calculando el número 0 de Fibonacci


3

Verificamos que calculamos cada número una sola vez.

#### Verificar tipo de los parámetros de las funciones

Creemos un decorador que verifique el tipo de los parámetros recibidos en una función, y que en caso de no corresponder al tipo, lance una excepción.

Queremos poder especificar el tipo a verificar, por lo que el decorador debe recibir un parámetro con esa información. Y antes de ejecutar la función a decorar, debemos verificar todos los parámetros recibidos. Una solución al problema se muestra abajo.

In [10]:
from itertools import chain

def check_type(type_):
    def decorator(function):
        def wrapper(*args, **kwargs):
            for argument in chain(args, kwargs.values()):
                if not isinstance(argument, type_):
                    raise TypeError(f"El valor {argument} no es instancia de {type_}")
                return function(*args, **kwargs)
        return wrapper
    return decorator

Ahora, definamos una función para concatenar dos _strings_.

In [11]:
def concat_strings(str1, str2):
    return str1 + str2

print(concat_strings(1, 2))
print(concat_strings("Jesus", "Christ"))

3
JesusChrist


Este comportamiento no es el esperado de esta función, pues solo queríamos que permita concatenar strings. Vamos a decorarla.

In [12]:
@check_type(str)
def concat_strings(str1, str2):
    return str1 + str2

Si intentamos usar la función decorada para sumar enteros, tendremos un error. Este error no ocurria cuando usábamos la función no decorada

In [13]:
concat_strings(1, 2)

TypeError: El valor 1 no es instancia de <class 'str'>

Mientras tanto, si concatenamos _strings_ lo podremos hacer sin problemas:

In [14]:
concat_strings("Jesus", "Christ")

'JesusChrist'